In [2]:
import numpy as np
from matplotlib import pylab as plt
from skimage import img_as_float, img_as_ubyte,data, io, filters,feature,measure
from skimage.color import rgb2gray
import matplotlib.patheffects as pe
import skimage.morphology as mp
from skimage.morphology import square,disk
from skimage.filters import median
from scipy import ndimage
import builtins
from math import *




class Coin:
    result=''
    def __init__(self,radius,centre_x,centre_y,grey_whole,yellow_whole,grey_semiradius,yellow_semiradius,semiradius_brightness):  
        self.radius = radius  
        self.centre_x = centre_x 
        self.centre_y = centre_y 
        self.grey_whole = grey_whole 
        self.yellow_whole = yellow_whole 
        self.grey_semiradius =grey_semiradius
        self.yellow_semiradius = yellow_semiradius
        self.semiradius_brightness=semiradius_brightness
    def wypisz(self):
        print(self.__dict__)
        
#wyznaczanie wspolrzednych skrajnych punktow obwodu monety
def get_max_min(matrix):
        max_y,min_y,min_x,max_x=0,matrix[0][1],matrix[0][0],0
        for list in matrix:
            if list[1]>max_y:
                max_y=list[1]
            if list[1]<min_y:
                min_y=list[1]
            if list[0]>max_x:
                max_x=list[0]
            if list[0]<min_x:
                min_x=list[0]
        return ([max_y,min_y,min_x,max_x])
    
processed_images=0
enable_debugging=False

filenames=['images/('+str(i)+').jpg' for i in range(1,31)]


red,green,blue=56,35,8



for filename in filenames:
    fig = plt.figure(figsize = (15,7))
    print("processed file:",filename)
    img_colorful=img_as_float(io.imread(filename))
    img=img_as_float(rgb2gray(io.imread(filename)))
    img_original=img_as_float(io.imread(filename))
    
    if(enable_debugging==False):
        plt.subplot(1,2,1)
        plt.axis('off')
        plt.text(0, 0, filename.replace(".jpg","").replace('images/','')+" INPUT:", size=20,color='white',path_effects=[pe.withStroke(linewidth=4, foreground="green")])
        plt.imshow(img_original)
    
    for i in range(len(img_colorful)):
        for j in range(len(img_colorful[0])):
            if(red/255+img_colorful[i][j][0]>1):img_colorful[i][j][0]=1
            else:img_colorful[i][j][0]+=red/255
            if(green/255+img_colorful[i][j][1]>1):img_colorful[i][j][1]=1
            else:img_colorful[i][j][1]+=green/255
            if(img_colorful[i][j][2]+blue/255>1):img_colorful[i][j][2]=1
            else:img_colorful[i][j][2]+=blue/255
            
#------------------------czesc przygotowania obrazu do wykrywania konturow-----------
    test=img

    test=feature.canny(test,sigma=0.5)

    dilation_size=10
    
    test = mp.dilation(test,square(dilation_size))
    test = median(test, disk(14))
    test=  mp.erosion(test,square(dilation_size))
    test = mp.dilation(mp.erosion(test))
    test = mp.dilation(test,square(dilation_size))
    test = mp.dilation(test,square(dilation_size))
    test=ndimage.binary_fill_holes(test)
    test=  mp.erosion(test,square(40))
    

    test = mp.dilation(test,square(20))
    test = mp.dilation(mp.erosion(test))
    test=mp.erosion(test)
    
    contours = measure.find_contours(test,0.01)

#-------------------------------czesc wykrywania konturow---------------
    
    err_threshold=100

    num_of_objects=builtins.sum(1 for i in contours if len(i)>err_threshold)
    #print("number of coins:",num_of_objects)

    if(enable_debugging==False):
        plt.subplot(1,2,2)
        
    
    if num_of_objects==0: print("nie wykryto zadnych obiektow")
    else:
        coins_array = []
        for contour in contours:
            if(len(contour)>err_threshold):
                centre_x, centre_y=np.mean(contour, axis=0)
                minmax=get_max_min(contour)
                
                

                radius=np.mean([minmax[0]-centre_y,centre_y-minmax[1],centre_x-minmax[2],minmax[3]-centre_x])
                if(radius>180):
                    num_of_objects-=1
                    continue
                
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2)
                
                semiradius_brightness_tab=[]
                
                

                yellow_whole,grey_whole,yellow_semiradius,grey_semiradius=0,0,0,0
                for x in range(int(centre_x-radius),int(centre_x+radius)):
                    for y in range(int(centre_y-radius),int(centre_y+radius)):
                        if(x>=len(img_colorful) or y>=len(img_colorful[0])):break
                        diff=max([abs(img_colorful[x][y][0]-img_colorful[x][y][1]),abs(img_colorful[x][y][0]-img_colorful[x][y][2]),abs(img_colorful[x][y][1]-img_colorful[x][y][2])])
                        diff*=255
                        #operacje w obszarze calego konturu
                        if(sqrt((x-centre_x)**2+(y-centre_y)**2)<radius):
                            if(diff<65):grey_whole+=1
                            else: yellow_whole+=1
                        #operacje w obszarze polowy promienia
                        if(sqrt((x-centre_x)**2+(y-centre_y)**2)<radius/2.0):
                            semiradius_brightness_tab.append(img[x][y])
                            if(diff<65):grey_semiradius+=1
                            else: yellow_semiradius+=1
                
                #podanie procentowego stosunku yellow/grey
                yellow_whole,grey_whole=100*yellow_whole/(yellow_whole+grey_whole),100*grey_whole/(yellow_whole+grey_whole)
                yellow_semiradius,grey_semiradius=100*yellow_semiradius/(yellow_semiradius+grey_semiradius),100*grey_semiradius/(yellow_semiradius+grey_semiradius)

                coins_array.append(Coin(radius,centre_x,centre_y,grey_whole,yellow_whole,grey_semiradius,yellow_semiradius,np.mean(semiradius_brightness_tab)))

        radii=[i.radius for i in coins_array]

        biggest_diff=0
        for i in range(len(radii)):
            for j in range(i+1,len(radii)):
                if(abs(radii[i]-radii[j]))>biggest_diff:
                    biggest_diff=abs(radii[i]-radii[j])
        max_radius=max(radii)

        same_size_coins=False
        
        big_small_threshold=max_radius*0.72

        if(biggest_diff<25): 
            same_size_coins=True
        
        total_sum_value=0
    #----------------------etap interpretacji uzyskanych danych-------------
        for obj in coins_array:
            result,value='NIE ROZPOZNANO',0
            if(obj.yellow_whole>25):
                if(obj.grey_whole>13):
                    if(obj.yellow_semiradius>50 and obj.radius>big_small_threshold):result,value='5 ZŁOTYCH',5
                    elif(obj.yellow_semiradius<50 and obj.radius>big_small_threshold): result,value='2 ZŁOTE',2
                if(obj.yellow_whole>60 and obj.radius<big_small_threshold and result=="NIE ROZPOZNANO"): result,value='1 GROSZ',0.01
            if(same_size_coins==False):
                if(obj.grey_whole>50  and result=="NIE ROZPOZNANO"):
                    if(obj.radius<big_small_threshold): result,value='10 GROSZY',0.1
                    else: result,value='1 ZŁOTY',1
            elif (obj.grey_whole>20 and result=="NIE ROZPOZNANO"):
                result,value='N/A',0
            total_sum_value+=value
            obj.result=result 
    
    
        
        if(same_size_coins):
            recognized_coins=[i.result for i in coins_array]
            #print(recognized_coins)
            for obj in coins_array:
                if obj.result=="N/A":
                    if ("2 ZŁOTE" in recognized_coins) or ("5 ZŁOTYCH" in recognized_coins):
                        obj.result="1 ZŁOTY"
                        total_sum_value+=1
                    elif ("1 GROSZ" in recognized_coins):
                        obj.result="10 GROSZY"
                        total_sum_value+=0.1
                    elif(obj.semiradius_brightness<0.3):
                        obj.result="10 GROSZY"
                        total_sum_value+=0.1
                    else:
                        obj.result="1 ZŁOTY"
                        total_sum_value+=1
                    
                
    #-------------------wypisanie wyniku-------------------
        for obj in coins_array:
            if enable_debugging:
                plt.text(obj.centre_y, obj.centre_x, \
                    str("radius="+str(obj.radius)+"\nsemiradius_brightness"+str(round(obj.semiradius_brightness,2))+"\nyellow="+str(round(obj.yellow_whole,2))+"%\ngrey="+str(round(obj.grey_whole,2))+\
                        "%\nyellow_semiradius="+str(round(obj.yellow_semiradius,2))+"%\ngrey_semiradius="+str(round(obj.grey_semiradius,2))+\
                        "%\n"+obj.result), color='red',size=12)
                #pprint(vars(obj))
            else: plt.text(obj.centre_y, obj.centre_x, obj.result, size=11,color='white',path_effects=[pe.withStroke(linewidth=4, foreground="red")])
           
            plt.text(0, 0, "OUTPUT:\ntotal value="+str(round(total_sum_value,2))+"zł\nnumber of coins="+str(num_of_objects), size=20,color='white',path_effects=[pe.withStroke(linewidth=4, foreground="green")])

        #plt.show()
        plt.axis('off')
        plt.imshow(img_original)
        fig.savefig('output/'+filename)
        plt.close(fig)
        
        processed_images+=1



processed file: images/(1).jpg
processed file: images/(2).jpg
processed file: images/(3).jpg
processed file: images/(4).jpg
processed file: images/(5).jpg
processed file: images/(6).jpg
processed file: images/(7).jpg
processed file: images/(8).jpg
processed file: images/(9).jpg
processed file: images/(10).jpg
processed file: images/(11).jpg
processed file: images/(12).jpg
processed file: images/(13).jpg
processed file: images/(14).jpg
processed file: images/(15).jpg
processed file: images/(16).jpg
processed file: images/(17).jpg
processed file: images/(18).jpg
processed file: images/(19).jpg
processed file: images/(20).jpg
processed file: images/(21).jpg
processed file: images/(22).jpg
processed file: images/(23).jpg
processed file: images/(24).jpg
processed file: images/(25).jpg
processed file: images/(26).jpg
processed file: images/(27).jpg
processed file: images/(28).jpg
processed file: images/(29).jpg
processed file: images/(30).jpg
